In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
import nltk
import pyarabic.araby as araby
from pyarabic.araby import strip_tashkeel
import qalsadi.lemmatizer 
import qalsadi.analex as qa
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer 
from farasa.diacratizer import FarasaDiacritizer 
from farasa.segmenter import FarasaSegmenter 
from farasa.stemmer import FarasaStemmer

import keras
from  diacritization_evaluation import util

e:\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
words = []
sentences = []
sentences_with_tashkeel = []
with open('./Dataset/WordsWithoutTashkeel.txt', 'r', encoding='utf-8') as output_file:
    for word in output_file:
        words.append(word.strip())

with open('./Dataset/SentencesWithoutTashkeel.txt', 'r', encoding='utf-8') as output_file:
    for sentence in output_file:
        sentences.append(sentence.strip())

with open('./Dataset/sentences.txt', 'r', encoding='utf-8') as output_file:
    for sentence in output_file:
        sentences_with_tashkeel.append(sentence.strip())

print(words[0:10])
print(sentences[0])
print(sentences_with_tashkeel[0])


['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة']
قوله أو قطع الأول يده إلخ قال الزركشي ابن عرفة قوله بلفظ يقتضيه كإنكار غير حديث بالإسلام وجوب ما علم وجوبه من الدين ضرورة كإلقاء مصحف بقذر وشد زنار ابن عرفة قول ابن شاس أو بفعل يتضمنه هو كلبس الزنار وإلقاء المصحف في صريح النجاسة والسجود للصنم ونحو ذلك وسحر محمد قول مالك وأصحابه أن الساحر كافر بالله تعالى قال مالك هو كالزنديق إذا عمل السحر بنفسه قتل ولم يستتب
قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ هُوَ كَالزِّنْدِيقِ إذَا 

In [4]:
print(sentences_with_tashkeel[0])

قَوْلُهُ أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ الزَّرْكَشِيُّ ابْنُ عَرَفَةَ قَوْلُهُ بِلَفْظٍ يَقْتَضِيه كَإِنْكَارِ غَيْرِ حَدِيثٍ بِالْإِسْلَامِ وُجُوبَ مَا عُلِمَ وُجُوبُهُ مِنْ الدِّينِ ضَرُورَةً كَإِلْقَاءِ مُصْحَفٍ بِقَذَرٍ وَشَدِّ زُنَّارٍ ابْنُ عَرَفَةَ قَوْلُ ابْنِ شَاسٍ أَوْ بِفِعْلٍ يَتَضَمَّنُهُ هُوَ كَلُبْسِ الزُّنَّارِ وَإِلْقَاءِ الْمُصْحَفِ فِي صَرِيحِ النَّجَاسَةِ وَالسُّجُودِ لِلصَّنَمِ وَنَحْوِ ذَلِكَ وَسِحْرٍ مُحَمَّدٌ قَوْلُ مَالِكٍ وَأَصْحَابِهِ أَنَّ السَّاحِرَ كَافِرٌ بِاَللَّهِ تَعَالَى قَالَ مَالِكٌ هُوَ كَالزِّنْدِيقِ إذَا عَمِلَ السِّحْرَ بِنَفْسِهِ قُتِلَ وَلَمْ يُسْتَتَبْ


Creating a word-based tokenizer

In [5]:
# Create a tokenizer
words_tokenizer = Tokenizer()

# Fit the tokenizer on the list of words (treat each word as a separate "sentence")
words_tokenizer.fit_on_texts(sentences)

# Get the word index
word_index = words_tokenizer.word_index

# Tokenize the words
word_sequences = words_tokenizer.texts_to_sequences(sentences)

Testing the tokenizer on a sample sentence

In [6]:
# Create a sentence tokenizer
print((sentences[0].split(" "))) #This way works and the one used below as well
print(word_index["قوله"])
print(words_tokenizer.texts_to_sequences([sentences[0]]))

['قوله', 'أو', 'قطع', 'الأول', 'يده', 'إلخ', 'قال', 'الزركشي', 'ابن', 'عرفة', 'قوله', 'بلفظ', 'يقتضيه', 'كإنكار', 'غير', 'حديث', 'بالإسلام', 'وجوب', 'ما', 'علم', 'وجوبه', 'من', 'الدين', 'ضرورة', 'كإلقاء', 'مصحف', 'بقذر', 'وشد', 'زنار', 'ابن', 'عرفة', 'قول', 'ابن', 'شاس', 'أو', 'بفعل', 'يتضمنه', 'هو', 'كلبس', 'الزنار', 'وإلقاء', 'المصحف', 'في', 'صريح', 'النجاسة', 'والسجود', 'للصنم', 'ونحو', 'ذلك', 'وسحر', 'محمد', 'قول', 'مالك', 'وأصحابه', 'أن', 'الساحر', 'كافر', 'بالله', 'تعالى', 'قال', 'مالك', 'هو', 'كالزنديق', 'إذا', 'عمل', 'السحر', 'بنفسه', 'قتل', 'ولم', 'يستتب']
8
[[8, 4, 273, 91, 189, 47, 14, 901, 37, 458, 8, 1184, 3641, 41299, 39, 249, 4154, 475, 12, 163, 1738, 3, 158, 1175, 27872, 6099, 41300, 5827, 41301, 37, 458, 63, 37, 2652, 4, 1362, 24279, 43, 17883, 57670, 41302, 2961, 2, 1430, 2148, 9577, 57671, 1189, 18, 21681, 86, 63, 94, 3359, 6, 16422, 1190, 1185, 108, 14, 94, 43, 27873, 28, 520, 8288, 415, 264, 51, 57672]]


Creating a character-based with tashkeel tokenizer

In [30]:
sentences_new = []
with open('./Dataset/sentences_new_approach.txt', 'r', encoding='utf-8') as output_file:
    for sentence in output_file:
        sentences_new.append(sentence.strip())

char_tokenizer_with_tashkeel = Tokenizer(char_level=True, oov_token='UNK')
char_tokenizer_with_tashkeel.fit_on_texts(sentences_new)
char_index_with_tashkeel = char_tokenizer_with_tashkeel.word_index
char_sequences_with_tashkeel = char_tokenizer_with_tashkeel.texts_to_sequences(sentences_new)
print(char_sequences_with_tashkeel[0:5])
print(char_index_with_tashkeel)

[[20, 2, 11, 5, 6, 8, 13, 8, 3, 18, 2, 11, 5, 3, 20, 2, 41, 2, 16, 2, 3, 7, 6, 5, 18, 2, 11, 17, 6, 8, 3, 10, 2, 22, 2, 13, 8, 3, 28, 6, 2, 34, 5, 3, 20, 2, 7, 6, 2, 3, 7, 6, 42, 17, 15, 5, 23, 2, 37, 4, 10, 43, 3, 7, 14, 5, 12, 8, 3, 16, 2, 15, 2, 19, 2, 26, 2, 3, 20, 2, 11, 5, 6, 8, 13, 8, 3, 14, 4, 6, 2, 19, 5, 47, 31, 3, 10, 2, 20, 5, 21, 2, 38, 4, 10, 13, 3, 23, 2, 28, 4, 12, 5, 23, 2, 7, 15, 4, 3, 44, 2, 10, 5, 15, 4, 3, 24, 2, 22, 4, 10, 35, 31, 3, 14, 4, 7, 6, 5, 28, 4, 25, 5, 6, 2, 7, 9, 4, 3, 11, 8, 27, 8, 11, 14, 2, 3, 9, 2, 7, 3, 16, 8, 6, 4, 9, 2, 3, 11, 8, 27, 8, 11, 14, 8, 13, 8, 3, 9, 4, 12, 5, 3, 7, 6, 22, 36, 10, 12, 4, 3, 38, 2, 15, 8, 11, 15, 2, 26, 39, 3, 23, 2, 28, 4, 6, 5, 20, 2, 7, 45, 4, 3, 9, 8, 30, 5, 24, 2, 19, 31, 3, 14, 4, 20, 2, 29, 2, 15, 31, 3, 11, 2, 37, 2, 22, 36, 3, 42, 8, 12, 17, 7, 15, 31, 3, 7, 14, 5, 12, 8, 3, 16, 2, 15, 2, 19, 2, 26, 2, 3, 20, 2, 11, 5, 6, 8, 3, 7, 14, 5, 12, 4, 3, 37, 2, 7, 25, 31, 3, 18, 2, 11, 5, 3, 14, 4, 19, 4, 16, 5, 6, 31

Creating a character-based without tashkeel tokenizer

In [9]:
char_tokenizer_without_tashkeel = Tokenizer(char_level=True)
char_tokenizer_without_tashkeel.fit_on_texts(sentences)
char_index_without_tashkeel = char_tokenizer_without_tashkeel.word_index
char_sequences_without_tashkeel = char_tokenizer_without_tashkeel.texts_to_sequences(sentences)


In [28]:
char_sequences_without_tashkeel = char_tokenizer_without_tashkeel.texts_to_sequences(sentences)

In [31]:
print(char_sequences_without_tashkeel[0:1])
print(len(char_index_without_tashkeel.keys()))
print(char_index_without_tashkeel)
print(char_index_with_tashkeel)

[[14, 6, 2, 8, 1, 12, 6, 1, 14, 31, 11, 1, 3, 2, 12, 6, 2, 1, 5, 16, 8, 1, 22, 2, 27, 1, 14, 3, 2, 1, 3, 2, 32, 10, 17, 29, 5, 1, 3, 9, 7, 1, 11, 10, 13, 20, 1, 14, 6, 2, 8, 1, 9, 2, 13, 36, 1, 5, 14, 15, 30, 5, 8, 1, 17, 22, 7, 17, 3, 10, 1, 33, 5, 10, 1, 18, 16, 5, 28, 1, 9, 3, 2, 22, 19, 2, 3, 4, 1, 6, 21, 6, 9, 1, 4, 3, 1, 11, 2, 4, 1, 6, 21, 6, 9, 8, 1, 4, 7, 1, 3, 2, 16, 5, 7, 1, 30, 10, 6, 10, 20, 1, 17, 22, 2, 14, 3, 34, 1, 4, 24, 18, 13, 1, 9, 14, 23, 10, 1, 6, 29, 16, 1, 32, 7, 3, 10, 1, 3, 9, 7, 1, 11, 10, 13, 20, 1, 14, 6, 2, 1, 3, 9, 7, 1, 29, 3, 19, 1, 12, 6, 1, 9, 13, 11, 2, 1, 5, 15, 30, 4, 7, 8, 1, 8, 6, 1, 17, 2, 9, 19, 1, 3, 2, 32, 7, 3, 10, 1, 6, 22, 2, 14, 3, 34, 1, 3, 2, 4, 24, 18, 13, 1, 13, 5, 1, 24, 10, 5, 18, 1, 3, 2, 7, 21, 3, 19, 20, 1, 6, 3, 2, 19, 21, 6, 16, 1, 2, 2, 24, 7, 4, 1, 6, 7, 18, 6, 1, 23, 2, 17, 1, 6, 19, 18, 10, 1, 4, 18, 4, 16, 1, 14, 6, 2, 1, 4, 3, 2, 17, 1, 6, 12, 24, 18, 3, 9, 8, 1, 12, 7, 1, 3, 2, 19, 3, 18, 10, 1, 17, 3, 13, 10, 1, 9, 3, 

Adding tests for the character-based tokenizer

In [32]:
print(char_index_with_tashkeel)
print(char_tokenizer_with_tashkeel.texts_to_sequences(sentences_new[12].split(" ")))

{'UNK': 1, 'َ': 2, ' ': 3, 'ِ': 4, 'ْ': 5, 'ل': 6, 'ا': 7, 'ُ': 8, 'م': 9, 'ي': 10, 'و': 11, 'ن': 12, 'ه': 13, 'ب': 14, 'ر': 15, 'ع': 16, 'd': 17, 'أ': 18, 'ف': 19, 'ق': 20, 'ت': 21, 'د': 22, 'ك': 23, 'ح': 24, 'س': 25, 'ة': 26, 'ج': 27, 'إ': 28, 'ذ': 29, 'ص': 30, 'ٍ': 31, '،': 32, 'ى': 33, 'خ': 34, 'ث': 35, 'f': 36, 'ش': 37, 'ض': 38, 'ً': 39, 'ٌ': 40, 'ط': 41, 'ز': 42, 'e': 43, 'غ': 44, 'ء': 45, 'ئ': 46, 'ظ': 47, 'آ': 48, 'ؤ': 49, 'ّ': 50, 'c': 51, 'b': 52, 'a': 53}
[[20, 2, 7, 6, 8, 11, 7], [6, 4, 18, 2, 12, 17, 13, 8], [6, 2, 11, 5], [11, 2, 27, 2, 14, 2], [16, 2, 6, 2, 10, 5, 13, 4, 9, 5], [24, 2, 9, 5, 6, 8, 13, 2, 7], [6, 2, 14, 2, 41, 2, 6, 2, 21, 5], [7, 6, 30, 17, 6, 2, 7, 26, 8], [14, 4, 21, 2, 15, 5, 23, 4, 13, 2, 7]]


In [33]:
# New text data for testing
new_texts = ["مرحبا كيف حالك", "السلام عليكم"]

# Tokenize the new text data at the character level
sequences_new = char_tokenizer_with_tashkeel.texts_to_sequences(new_texts)

# Print the results
for text, sequence in zip(new_texts, sequences_new):
    print(f"Original Text: {text}")
    print(f"Tokenized Sequence: {sequence}\n")

Original Text: مرحبا كيف حالك
Tokenized Sequence: [9, 15, 24, 14, 7, 3, 23, 10, 19, 3, 24, 7, 6, 23]

Original Text: السلام عليكم
Tokenized Sequence: [7, 6, 25, 6, 7, 9, 3, 16, 6, 10, 23, 9]



Addding Padding to the tokenizers

In [43]:

# Assuming word_sequences and char_sequences are the output of the tokenizers
word_sequences = words_tokenizer.texts_to_sequences(sentences)
char_sequences = char_tokenizer_with_tashkeel.texts_to_sequences(sentences_new)

# Add padding
word_sequences_padded = pad_sequences(word_sequences, padding='post')
char_sequences_with_tashkeel_padded = pad_sequences(char_sequences, padding='post')
char_sequences_without_tashkeel_padded = pad_sequences(char_sequences_without_tashkeel, padding='post')

In [44]:
print(len(char_sequences[5]))
print(len(char_sequences_with_tashkeel_padded[5]))

164
4263


In [45]:
# Save the tokenized sequences
with open('./pickles/word_sequences.pkl', 'wb') as file:
    pickle.dump(word_sequences_padded, file)

with open('./pickles/char_sequences_with_tashkeel.pkl', 'wb') as file:
    pickle.dump(char_sequences_with_tashkeel_padded, file)

with open('./pickles/char_sequences_without_tashkeel.pkl', 'wb') as file:
    pickle.dump(char_sequences_without_tashkeel_padded, file)

TESTING labels


In [46]:
# sentences_with_tashkeel
tashkeel_list = []
for sentence in sentences_with_tashkeel:
    text, txt_list, harakat_list = util.extract_haraqat(sentence)   
    for i in range(len(harakat_list)):
        if len(harakat_list[i]) == 2:
            if '\u0651\u064B' in harakat_list[i]:
                harakat_list[i] = 'a'
            if '\u0651\u064C' in harakat_list[i]:
                harakat_list[i] = 'b'
            if '\u0651\u064D' in harakat_list[i]:
                harakat_list[i] = 'c'
            if '\u0651\u064E' in harakat_list[i]:
                harakat_list[i] = 'd'
            if '\u0651\u064F' in harakat_list[i]:
                harakat_list[i] = 'e'
            if '\u0651\u0650' in harakat_list[i]:
                harakat_list[i] = 'f'

    tashkeel_list.append(harakat_list)



In [47]:
tashkeel_sequences = char_tokenizer_with_tashkeel.texts_to_sequences(tashkeel_list)
tashkeel_sequences_padded = pad_sequences(tashkeel_sequences, padding='post')
with open('tashkeel_sequences.pkl', 'wb') as file:
    pickle.dump(tashkeel_sequences_padded, file)

In [48]:
print(tashkeel_list[0])

['َ', 'ْ', 'ُ', 'ُ', '', 'َ', 'ْ', '', 'َ', 'َ', 'َ', '', '', 'ْ', 'َ', 'd', 'ُ', '', 'َ', 'َ', 'ُ', '', '', 'َ', 'ْ', '', 'َ', '', 'َ', '', '', '', 'd', 'ْ', 'َ', 'ِ', 'e', '', '', 'ْ', 'ُ', '', 'َ', 'َ', 'َ', 'َ', '', 'َ', 'ْ', 'ُ', 'ُ', '', 'ِ', 'َ', 'ْ', 'ٍ', '', 'َ', 'ْ', 'َ', 'ِ', '', '', '', 'َ', 'ِ', 'ْ', 'َ', '', 'ِ', '', 'َ', 'ْ', 'ِ', '', 'َ', 'ِ', '', 'ٍ', '', 'ِ', '', 'ْ', 'ِ', 'ْ', 'َ', '', 'ِ', '', 'ُ', 'ُ', '', 'َ', '', 'َ', '', '', 'ُ', 'ِ', 'َ', '', 'ُ', 'ُ', '', 'ُ', 'ُ', '', 'ِ', 'ْ', '', '', '', 'f', '', 'ِ', '', 'َ', 'ُ', '', 'َ', 'ً', '', 'َ', 'ِ', 'ْ', 'َ', '', 'ِ', '', 'ُ', 'ْ', 'َ', 'ٍ', '', 'ِ', 'َ', 'َ', 'ٍ', '', 'َ', 'َ', 'f', '', 'ُ', 'd', '', 'ٍ', '', '', 'ْ', 'ُ', '', 'َ', 'َ', 'َ', 'َ', '', 'َ', 'ْ', 'ُ', '', '', 'ْ', 'ِ', '', 'َ', '', 'ٍ', '', 'َ', 'ْ', '', 'ِ', 'ِ', 'ْ', 'ٍ', '', 'َ', 'َ', 'َ', 'd', 'ُ', 'ُ', '', 'ُ', 'َ', '', 'َ', 'ُ', 'ْ', 'ِ', '', '', '', 'e', 'd', '', 'ِ', '', 'َ', 'ِ', 'ْ', 'َ', '', 'ِ', '', '', 'ْ', 'ُ', 'ْ', 'َ', 'ِ', '', 'ِ', 

In [49]:

print(len(tashkeel_sequences_padded[0]))
print(len(tashkeel_sequences_padded[0]))

2592
2592


In [54]:
print(len(tashkeel_sequences[1]))
print(len(char_sequences_without_tashkeel[1]))

print(char_tokenizer_with_tashkeel.sequences_to_texts([tashkeel_sequences[0]]))
print(char_tokenizer_with_tashkeel.sequences_to_texts([char_sequences_without_tashkeel[0]]))

425
425
['َ ْ ُ ُ UNK َ ْ UNK َ َ َ UNK UNK ْ َ d ُ UNK َ َ ُ UNK UNK َ ْ UNK َ UNK َ UNK UNK UNK d ْ َ ِ e UNK UNK ْ ُ UNK َ َ َ َ UNK َ ْ ُ ُ UNK ِ َ ْ ٍ UNK َ ْ َ ِ UNK UNK UNK َ ِ ْ َ UNK ِ UNK َ ْ ِ UNK َ ِ UNK ٍ UNK ِ UNK ْ ِ ْ َ UNK ِ UNK ُ ُ UNK َ UNK َ UNK UNK ُ ِ َ UNK ُ ُ UNK ُ ُ UNK ِ ْ UNK UNK UNK f UNK ِ UNK َ ُ UNK َ ً UNK َ ِ ْ َ UNK ِ UNK ُ ْ َ ٍ UNK ِ َ َ ٍ UNK َ َ f UNK ُ d UNK ٍ UNK UNK ْ ُ UNK َ َ َ َ UNK َ ْ ُ UNK UNK ْ ِ UNK َ UNK ٍ UNK َ ْ UNK ِ ِ ْ ٍ UNK َ َ َ d ُ ُ UNK ُ َ UNK َ ُ ْ ِ UNK UNK UNK e d UNK ِ UNK َ ِ ْ َ UNK ِ UNK UNK ْ ُ ْ َ ِ UNK ِ UNK UNK َ ِ UNK ِ UNK UNK UNK d َ UNK َ ِ UNK َ UNK UNK e ُ UNK ِ UNK ِ UNK d َ ِ UNK َ َ ْ ِ UNK َ ِ َ UNK َ ِ ْ ٍ UNK ُ َ d ٌ UNK َ ْ ُ UNK َ UNK ِ ٍ UNK َ َ ْ َ UNK ِ ِ UNK َ d UNK UNK UNK d UNK ِ َ UNK َ UNK ِ ٌ UNK ِ َ UNK d ِ UNK َ َ UNK َ UNK UNK َ UNK َ UNK َ UNK ِ ٌ UNK ُ َ UNK َ UNK UNK f ْ ِ UNK ِ UNK UNK َ UNK UNK َ ِ َ UNK UNK UNK f ْ َ UNK ِ َ ْ ِ ِ UNK ُ ِ َ UNK َ َ ْ UNK ُ ْ َ َ ْ']
['ب ل َ ُ UNK ن ل 

# Tokenizing Tashkeel only

In [56]:
tashkeel_tokenizer = Tokenizer(char_level=True, oov_token='UNK')
tashkeel_tokenizer.fit_on_texts(tashkeel_list)
tashkeel_index = tashkeel_tokenizer.word_index
tashkeel_list_sequences = tashkeel_tokenizer.texts_to_sequences(tashkeel_list)

tashkeel_list_sequences_padded = pad_sequences(tashkeel_list_sequences, padding='post')

In [57]:
print(tashkeel_list_sequences[0:3])
print(tashkeel_index)
print(len(tashkeel_list_sequences_padded[0]))

[[3, 5, 6, 6, 2, 3, 5, 2, 3, 3, 3, 2, 2, 5, 3, 7, 6, 2, 3, 3, 6, 2, 2, 3, 5, 2, 3, 2, 3, 2, 2, 2, 7, 5, 3, 4, 12, 2, 2, 5, 6, 2, 3, 3, 3, 3, 2, 3, 5, 6, 6, 2, 4, 3, 5, 8, 2, 3, 5, 3, 4, 2, 2, 2, 3, 4, 5, 3, 2, 4, 2, 3, 5, 4, 2, 3, 4, 2, 8, 2, 4, 2, 5, 4, 5, 3, 2, 4, 2, 6, 6, 2, 3, 2, 3, 2, 2, 6, 4, 3, 2, 6, 6, 2, 6, 6, 2, 4, 5, 2, 2, 2, 9, 2, 4, 2, 3, 6, 2, 3, 10, 2, 3, 4, 5, 3, 2, 4, 2, 6, 5, 3, 8, 2, 4, 3, 3, 8, 2, 3, 3, 9, 2, 6, 7, 2, 8, 2, 2, 5, 6, 2, 3, 3, 3, 3, 2, 3, 5, 6, 2, 2, 5, 4, 2, 3, 2, 8, 2, 3, 5, 2, 4, 4, 5, 8, 2, 3, 3, 3, 7, 6, 6, 2, 6, 3, 2, 3, 6, 5, 4, 2, 2, 2, 12, 7, 2, 4, 2, 3, 4, 5, 3, 2, 4, 2, 2, 5, 6, 5, 3, 4, 2, 4, 2, 2, 3, 4, 2, 4, 2, 2, 2, 7, 3, 2, 3, 4, 2, 3, 2, 2, 12, 6, 2, 4, 2, 4, 2, 7, 3, 4, 2, 3, 3, 5, 4, 2, 3, 4, 3, 2, 3, 4, 5, 8, 2, 6, 3, 7, 11, 2, 3, 5, 6, 2, 3, 2, 4, 8, 2, 3, 3, 5, 3, 2, 4, 4, 2, 3, 7, 2, 2, 2, 7, 2, 4, 3, 2, 3, 2, 4, 11, 2, 4, 3, 2, 7, 4, 2, 3, 3, 2, 3, 2, 2, 3, 2, 3, 2, 3, 2, 4, 11, 2, 6, 3, 2, 3, 2, 2, 9, 5, 4, 2, 4, 2, 2, 3, 2, 2

In [58]:
with open('./pickles/tashkeel_sequences.pkl', 'wb') as file:
    pickle.dump(tashkeel_list_sequences_padded, file)

In [59]:
print(tashkeel_list_sequences[0])

[3, 5, 6, 6, 2, 3, 5, 2, 3, 3, 3, 2, 2, 5, 3, 7, 6, 2, 3, 3, 6, 2, 2, 3, 5, 2, 3, 2, 3, 2, 2, 2, 7, 5, 3, 4, 12, 2, 2, 5, 6, 2, 3, 3, 3, 3, 2, 3, 5, 6, 6, 2, 4, 3, 5, 8, 2, 3, 5, 3, 4, 2, 2, 2, 3, 4, 5, 3, 2, 4, 2, 3, 5, 4, 2, 3, 4, 2, 8, 2, 4, 2, 5, 4, 5, 3, 2, 4, 2, 6, 6, 2, 3, 2, 3, 2, 2, 6, 4, 3, 2, 6, 6, 2, 6, 6, 2, 4, 5, 2, 2, 2, 9, 2, 4, 2, 3, 6, 2, 3, 10, 2, 3, 4, 5, 3, 2, 4, 2, 6, 5, 3, 8, 2, 4, 3, 3, 8, 2, 3, 3, 9, 2, 6, 7, 2, 8, 2, 2, 5, 6, 2, 3, 3, 3, 3, 2, 3, 5, 6, 2, 2, 5, 4, 2, 3, 2, 8, 2, 3, 5, 2, 4, 4, 5, 8, 2, 3, 3, 3, 7, 6, 6, 2, 6, 3, 2, 3, 6, 5, 4, 2, 2, 2, 12, 7, 2, 4, 2, 3, 4, 5, 3, 2, 4, 2, 2, 5, 6, 5, 3, 4, 2, 4, 2, 2, 3, 4, 2, 4, 2, 2, 2, 7, 3, 2, 3, 4, 2, 3, 2, 2, 12, 6, 2, 4, 2, 4, 2, 7, 3, 4, 2, 3, 3, 5, 4, 2, 3, 4, 3, 2, 3, 4, 5, 8, 2, 6, 3, 7, 11, 2, 3, 5, 6, 2, 3, 2, 4, 8, 2, 3, 3, 5, 3, 2, 4, 4, 2, 3, 7, 2, 2, 2, 7, 2, 4, 3, 2, 3, 2, 4, 11, 2, 4, 3, 2, 7, 4, 2, 3, 3, 2, 3, 2, 2, 3, 2, 3, 2, 3, 2, 4, 11, 2, 6, 3, 2, 3, 2, 2, 9, 5, 4, 2, 4, 2, 2, 3, 2, 2,